In [1]:
import numpy as np

In [11]:
import tensorflow as tf
import sklearn

In [4]:
train_x = []
train_y = []
count = 0
with open('CIKM2017_train/train.txt') as fhdl:
    for line in fhdl:
        count += 1
        if count % 50 == 0:
            print (float(count) / 10000,end=' '),
        linenum,label,datas = line.strip().split(',')
        label = float(label)
        datas = np.asarray(datas.split(' '),dtype=np.int8)
        train_x.append(datas)
        train_y.append(label)

0.005 0.01 0.015 0.02 0.025 0.03 0.035 0.04 0.045 0.05 0.055 0.06 0.065 0.07 0.075 0.08 0.085 0.09 0.095 0.1 0.105 0.11 0.115 0.12 0.125 0.13 0.135 0.14 0.145 0.15 0.155 0.16 0.165 0.17 0.175 0.18 0.185 0.19 0.195 0.2 0.205 0.21 0.215 0.22 0.225 0.23 0.235 0.24 0.245 0.25 0.255 0.26 0.265 0.27 0.275 0.28 0.285 0.29 0.295 0.3 0.305 0.31 0.315 0.32 0.325 0.33 0.335 0.34 0.345 0.35 0.355 0.36 0.365 0.37 0.375 0.38 0.385 0.39 0.395 0.4 0.405 0.41 0.415 0.42 0.425 0.43 0.435 0.44 0.445 0.45 0.455 0.46 0.465 0.47 0.475 0.48 0.485 0.49 0.495 0.5 0.505 0.51 0.515 0.52 0.525 0.53 0.535 0.54 0.545 0.55 0.555 0.56 0.565 0.57 0.575 0.58 0.585 0.59 0.595 0.6 0.605 0.61 0.615 0.62 0.625 0.63 0.635 0.64 0.645 0.65 0.655 0.66 0.665 0.67 0.675 0.68 0.685 0.69 0.695 0.7 0.705 0.71 0.715 0.72 0.725 0.73 0.735 0.74 0.745 0.75 0.755 0.76 0.765 0.77 0.775 0.78 0.785 0.79 0.795 0.8 0.805 0.81 0.815 0.82 0.825 0.83 0.835 0.84 0.845 0.85 0.855 0.86 0.865 0.87 0.875 0.88 0.885 0.89 0.895 0.9 0.905 0.91 0.915 0.

In [10]:
len(train_x),len(train_y)

(10000, 10000)

In [56]:
train_x,train_y,test_x,test_y = train_x[:9000],train_y[:9000],train_x[9000:],train_y[9000:]

In [57]:
train_x.shape,len(train_y),test_x.shape,len(test_y)

((9000, 612060), 9000, (1000, 612060), 1000)

In [58]:
train_x = np.asarray(train_x,dtype=np.int8)
test_x = np.asarray(test_x,dtype=np.int8)

In [ ]:
import pickle
pickle.dump((train_x,train_y),open('data.pkl','w'))

In [9]:
import sys
sys.getsizeof(train_x[0]) * 10000

6121560000

In [61]:
feed_size = 100
n_input = 101 * 101 * 60
X = tf.placeholder(tf.int8,[None,n_input])
W = tf.Variable(tf.zeros([n_input,1]),tf.float32)
b = tf.Variable([0.0],tf.float32)
y = tf.add(tf.matmul(tf.to_float(X),W),b)
y_ = tf.placeholder(tf.float32)
resm = tf.reduce_sum(tf.square(tf.transpose(y) - y_))

train_resm = tf.sqrt(tf.div(tf.reduce_sum(tf.square(tf.transpose(y) - y_)), feed_size))
real_resm = tf.sqrt(tf.div(tf.reduce_sum(tf.square(tf.transpose(y) - y_)), 1000))

train_step = tf.train.GradientDescentOptimizer(0.00000000000001).minimize(resm)
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

for i in range(500):
    indexbegin = feed_size * i % 9000
    batch_xs,batch_ys = train_x[indexbegin:indexbegin + feed_size,:],train_y[indexbegin:indexbegin + feed_size]
    if i % 10 == 0:
        print('itertion %s' % (i),end=' ')
        curr_loss = sess.run(train_resm,feed_dict={X:batch_xs,y_:batch_ys})
        print ("train loss: %s" % (curr_loss),end=' ')
        curr_loss = sess.run(real_resm,feed_dict={X:test_x,y_:test_y})
        print ("test loss: %s" % (curr_loss))
    sess.run(train_step,feed_dict={X:batch_xs,y_:batch_ys})

Instructions for updating:
Use `tf.global_variables_initializer` instead.
itertion 0 train loss: 6.82919 test loss: 17.9016
itertion 10 train loss: 29.4725 test loss: 17.6738
itertion 20 train loss: 20.2323 test loss: 17.4329
itertion 30 train loss: 26.3909 test loss: 17.2561
itertion 40 train loss: 17.5952 test loss: 17.1334
itertion 50 train loss: 13.4382 test loss: 16.9883
itertion 60 train loss: 23.0868 test loss: 16.8077
itertion 70 train loss: 29.7851 test loss: 16.6406
itertion 80 train loss: 24.7055 test loss: 16.5373
itertion 90 train loss: 5.89994 test loss: 16.4029
itertion 100 train loss: 28.3697 test loss: 16.2601
itertion 110 train loss: 18.989 test loss: 16.0822
itertion 120 train loss: 24.8171 test loss: 15.9626
itertion 130 train loss: 16.8546 test loss: 15.8844
itertion 140 train loss: 12.6082 test loss: 15.7857
itertion 150 train loss: 21.327 test loss: 15.6634
itertion 160 train loss: 28.5929 test loss: 15.5514
itertion 170 train loss: 23.9622 test loss: 15.4847
ite

In [60]:
feed_size = 100
n_input = 101 * 101 * 60
X = tf.placeholder(tf.int8,[None,n_input])
W = tf.Variable(tf.zeros([n_input,1]),tf.float32)
b = tf.Variable([0.0],tf.float32)
y = tf.add(tf.matmul(tf.to_float(X),W),b)
y_ = tf.placeholder(tf.float32)
resm = tf.reduce_sum(tf.square(tf.transpose(y) - y_))

train_resm = tf.sqrt(tf.div(tf.reduce_sum(tf.square(tf.transpose(y) - y_)), feed_size))
real_resm = tf.sqrt(tf.div(tf.reduce_sum(tf.square(tf.transpose(y) - y_)), 1000))

train_step = tf.train.GradientDescentOptimizer(0.00000000000003).minimize(resm)
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

for i in range(300):
    indexbegin = feed_size * i % 9000
    batch_xs,batch_ys = train_x[indexbegin:indexbegin + feed_size,:],train_y[indexbegin:indexbegin + feed_size]
    if i % 10 == 0:
        print('itertion %s' % (i),end=' ')
        curr_loss = sess.run(train_resm,feed_dict={X:batch_xs,y_:batch_ys})
        print ("train loss: %s" % (curr_loss),end=' ')
        curr_loss = sess.run(real_resm,feed_dict={X:test_x,y_:test_y})
        print ("test loss: %s" % (curr_loss))
    sess.run(train_step,feed_dict={X:batch_xs,y_:batch_ys})

Instructions for updating:
Use `tf.global_variables_initializer` instead.
itertion 0 train loss: 6.82919 test loss: 17.9016
itertion 10 train loss: 29.1683 test loss: 17.2526
itertion 20 train loss: 19.4203 test loss: 16.6113
itertion 30 train loss: 25.0982 test loss: 16.1989
itertion 40 train loss: 16.8922 test loss: 15.9486
itertion 50 train loss: 12.5061 test loss: 15.6573
itertion 60 train loss: 20.6948 test loss: 15.3317
itertion 70 train loss: 27.9408 test loss: 15.0655
itertion 80 train loss: 23.4719 test loss: 14.9267
itertion 90 train loss: 5.81648 test loss: 14.7414
itertion 100 train loss: 26.5748 test loss: 14.6321
itertion 110 train loss: 17.6394 test loss: 14.4432
itertion 120 train loss: 22.4769 test loss: 14.3649
itertion 130 train loss: 15.5398 test loss: 14.3329
itertion 140 train loss: 12.1298 test loss: 14.2845
itertion 150 train loss: 17.5755 test loss: 14.2396
itertion 160 train loss: 25.7794 test loss: 14.217
itertion 170 train loss: 22.2046 test loss: 14.2118
it

In [62]:
train_x = np.concatenate((train_x,test_x))
train_y = train_y + test_y

In [63]:
train_x = np.asarray(train_x,dtype=np.int8)

In [64]:
train_x.shape,len(train_y)

((10000, 612060), 10000)

In [65]:
del test_x
del test_y

In [66]:
test_x,test_y = train_x[9000:],train_y[9000:]

In [67]:
feed_size = 100
n_input = 101 * 101 * 60
X = tf.placeholder(tf.int8,[None,n_input])
W = tf.Variable(tf.zeros([n_input,1]),tf.float32)
b = tf.Variable([0.0],tf.float32)
y = tf.add(tf.matmul(tf.to_float(X),W),b)
y_ = tf.placeholder(tf.float32)
resm = tf.reduce_sum(tf.square(tf.transpose(y) - y_))

train_resm = tf.sqrt(tf.div(tf.reduce_sum(tf.square(tf.transpose(y) - y_)), feed_size))
real_resm = tf.sqrt(tf.div(tf.reduce_sum(tf.square(tf.transpose(y) - y_)), 1000))

train_step = tf.train.GradientDescentOptimizer(0.00000000000003).minimize(resm)
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

for i in range(170):
    indexbegin = feed_size * i % 10000
    batch_xs,batch_ys = train_x[indexbegin:indexbegin + feed_size,:],train_y[indexbegin:indexbegin + feed_size]
    if i % 10 == 0:
        print('itertion %s' % (i),end=' ')
        #curr_loss = sess.run(train_resm,feed_dict={X:batch_xs,y_:batch_ys})
        #print ("train loss: %s" % (curr_loss),end=' ')
        curr_loss = sess.run(real_resm,feed_dict={X:test_x,y_:test_y})
        print ("test loss: %s" % (curr_loss))
    sess.run(train_step,feed_dict={X:batch_xs,y_:batch_ys})

Instructions for updating:
Use `tf.global_variables_initializer` instead.
itertion 0 test loss: 17.9016
itertion 10 test loss: 17.2526
itertion 20 test loss: 16.6113
itertion 30 test loss: 16.1989
itertion 40 test loss: 15.9486
itertion 50 test loss: 15.6573
itertion 60 test loss: 15.3317
itertion 70 test loss: 15.0655
itertion 80 test loss: 14.9267
itertion 90 test loss: 14.7414
itertion 100 test loss: 14.6425
itertion 110 test loss: 14.551
itertion 120 test loss: 14.3846
itertion 130 test loss: 14.3192
itertion 140 test loss: 14.2939
itertion 150 test loss: 14.2549
itertion 160 test loss: 14.2206


In [68]:
del train_x
del train_y
del test_x
del test_y

In [69]:
predict_x = []
count = 0
with open('CIKM2017_testA/testA.txt') as fhdl:
    for line in fhdl:
        count += 1
        if count % 50 == 0:
            print (float(count) / 10000,end=' '),
        linenum,label,datas = line.strip().split(',')
        label = float(label)
        datas = np.asarray(datas.split(' '),dtype=np.int8)
        predict_x.append(datas)

0.005 0.01 0.015 0.02 0.025 0.03 0.035 0.04 0.045 0.05 0.055 0.06 0.065 0.07 0.075 0.08 0.085 0.09 0.095 0.1 0.105 0.11 0.115 0.12 0.125 0.13 0.135 0.14 0.145 0.15 0.155 0.16 0.165 0.17 0.175 0.18 0.185 0.19 0.195 0.2 

In [70]:
predict_y = sess.run(y,feed_dict={X:np.asarray(predict_x,dtype=np.int8)})

In [71]:
predict_y

array([[  7.89219189],
       [ 10.65508747],
       [ 12.67513371],
       ..., 
       [  5.09071112],
       [  2.88096619],
       [  3.77302599]], dtype=float32)

In [72]:
import pandas as pd

In [73]:
predict_y.shape

(2000, 1)

In [74]:
predict_y[0][0]

7.8921919

In [75]:
with open('answer_4_11.csv','w') as whdl:
    for value in predict_y:
        whdl.write("%s\n" % (value[0]))